In [1]:
using FileIO
using Images
using Statistics
using Flux: onehotbatch, onecold, crossentropy, binarycrossentropy
using Flux
using Random
using DelimitedFiles
using ImageView, Images, ImageSegmentation

include("funcionesUtiles.jl");

┌ Info: CUDAnative.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAnative /home/manuc/.julia/packages/CUDAnative/JfXpo/src/CUDAnative.jl:146
Gtk-Message: 13:39:23.178: Failed to load module "canberra-gtk-module"
Gtk-Message: 13:39:23.179: Failed to load module "canberra-gtk-module"


ErrorException: could not open file /home/manuc/Documentos/AA/practice/rna/funcionesUtiles.jl

In [2]:
function get_main_color(img, qValue1, qValue2, filter)
  
    matrizBooleana = filter.(img);
    matrizBooleana = closing(matrizBooleana)
    
    labelArray = ImageMorphology.label_components(matrizBooleana);
    
    boundingBoxes = ImageMorphology.component_boxes(labelArray);
    tamanos = ImageMorphology.component_lengths(labelArray); #Tamaño del grupo 
    pixeles = ImageMorphology.component_indices(labelArray);
    pixeles = ImageMorphology.component_subscripts(labelArray);
    centroides = ImageMorphology.component_centroids(labelArray);
    
    min_size = (size(img)[1]*size(img)[2]/600);
    
    etiquetasEliminar = findall(tamanos .<= min_size) .- 1; # Importate el -1, porque la primera etiqueta es la 0

    matrizBooleana = [!in(etiqueta,etiquetasEliminar) && (etiqueta!=0) for etiqueta in labelArray];
    
    labelArray = ImageMorphology.label_components(matrizBooleana);
    
    centroides = ImageMorphology.component_centroids(labelArray)[2:end];

    boundingBoxes = ImageMorphology.component_boxes(labelArray)[2:end];
    
    i = 1;
    R1 = Array{Float64,1}(undef, size(boundingBoxes));
    G1 = Array{Float64,1}(undef, size(boundingBoxes));
    B1 = Array{Float64,1}(undef, size(boundingBoxes));
    
    img_CHW = channelview(img);
    
    i = 1;
    for boundingBox in boundingBoxes
        x1 = boundingBox[1][1];
        y1 = boundingBox[1][2];
        x2 = boundingBox[2][1];
        y2 = boundingBox[2][2];

        R1[i] = quantile(skipmissing(img_CHW[1,x1:x2,y1:y2]), qValue1);
        G1[i] = quantile(skipmissing(img_CHW[2,x1:x2,y1:y2]), qValue1);
        B1[i] = quantile(skipmissing(img_CHW[3,x1:x2,y1:y2]), qValue1);
        i= i + 1;
    end;
    
    if(maximum(labelArray) == 0)
        return (quantile(skipmissing(red.(img)), qValue2), quantile(skipmissing(green.(img)), qValue2), quantile(skipmissing(blue.(img)),qValue2))
    else 
        return (quantile(skipmissing(R1), qValue2), quantile(skipmissing(G1), qValue2), quantile(skipmissing(B1),qValue2))
    end
end

get_main_color (generic function with 1 method)

In [3]:
esPixelRojo(pixel::RGBA, dRV=0.3, dRA=0.3) = (pixel.r > pixel.g + dRV) && (pixel.r > pixel.b + dRA);
esPixelRojo(pixel::RGB, dRV=0.3, dRA=0.3) = (pixel.r > pixel.g + dRV) && (pixel.r > pixel.b + dRA);
esPixelBlanco(pixel::RGB, dr=0.3, dg=0.3, db=0.3) = (pixel.r + dr >= 1) && (pixel.g + dg >= 1) && (pixel.b + db >= 1);
esPixelBlanco(pixel::RGBA, dr=0.3, dg=0.3, db=0.3) = (pixel.r + dr >= 1) && (pixel.g + dg >= 1) && (pixel.b + db >= 1);

In [2]:
function process_image(image)
    img = copy(image);
    img_CHW = channelview(img);
    
    (R1, G1, B1) = get_main_color(img, 0.99, 0.99, esPixelRojo)
    (R2, G2, B2) = get_main_color(img, 0.75, 0.95, esPixelBlanco)

    patron = [
        mean(img_CHW[1,:,:]), 
        mean(img_CHW[2,:,:]), 
        mean(img_CHW[3,:,:]),
        std(img_CHW[1,:,:]), 
        std(img_CHW[2,:,:]), 
        std(img_CHW[3,:,:]),
        R1,
        G1,
        B1,
        R2,
        G2,
        B2];
        
    return patron;
end;

In [4]:
function prepare_data(dir, n_inputs, output, patronesEnFilas = true)
    directorio = readdir(dir);
    
    salidas_deseadas = zeros(1, size(directorio,1));
    patrones = Array{Float64,2}(undef, size(directorio,1),n_inputs);
    
    i = 1
    
    if(patronesEnFilas)
        for file in directorio 
            image = load(string(dir,"/",file));
            patrones[i,:] = process_image(image)[:];
            salidas_deseadas[i] = output;
            i = i + 1
            
        end;
    else
        for file in directorio 
            image = load(string(dir,"/",file));
            patrones[:,i] = process_image(image)[:];
            salidas_deseadas[i] = output;
            i = i + 1
        end;
    end

    return patrones, salidas_deseadas;
end;

In [5]:
function get_orange_white_boolean(imagen)
    
    diferenciaRojoVerde = 0.3; 
    diferenciaRojoAzul = 0.3;
    
    canalRojo = red.(imagen); 
    canalVerde = green.(imagen); 
    canalAzul = blue.(imagen);
    
    white_chanel = (zeros(size(canalRojo)[1], size(canalRojo)[2]).+0.6);
    white_chanel1 = (zeros(size(canalRojo)[1], size(canalRojo)[2]).+0.9);
    
    mb_w =  (canalRojo.>white_chanel.+ mean(canalRojo)) .& 
            (canalVerde.>white_chanel.- mean(canalVerde)) .& 
            (canalAzul.>white_chanel.- mean(canalAzul));
    
    mb_w1 =  (canalRojo.>white_chanel1) .& 
            (canalVerde.>white_chanel1) .& 
            (canalAzul.>white_chanel1);
    
    mb_r =  (canalRojo.>(canalVerde.+diferenciaRojoVerde)) .& 
            (canalRojo.>(canalAzul.+diferenciaRojoAzul));
    
    matrizBooleana = mb_r .| mb_w1 #.| mb_w1; 
    
    return matrizBooleana;
end;

In [6]:
function check_region(x1, y1, x2, y2, imagen, modelo, nImputs, wall)
    img_v = @view imagen[x1:x2, y1:y2]
    img_CHW = channelview(img_v)

    xtrain = convert(Array{Float64}, process_image(img_v));
    
    xtrain_aux =  reshape(xtrain,1,nImputs);
    if(modelo.predict(xtrain_aux)[1] == 1.0)
        imagen[x1:x2, y1] .= RGB(0,1,0);
        imagen[x1:x2, y2] .= RGB(0,1,0);
        imagen[x1, y1:y2] .= RGB(0,1,0);
        imagen[x2, y1:y2] .= RGB(0,1,0);
    else
        imagen[x1:x2, y1] .= RGB(1,0,0);
        imagen[x1:x2, y2] .= RGB(1,0,0);
        imagen[x1, y1:y2] .= RGB(1,0,0);
        imagen[x2, y1:y2] .= RGB(1,0,0);
    end
end

check_region (generic function with 1 method)

In [39]:
function train_model(numEntradasRNA, numSalidasRNA, ArquitecturaRNA, patrones, salidasDeseadas)

    PorcentajeValidacion = 0.2;
    PorcentajeTest = 0.2;

    NumMaxCiclosEntrenamiento = 1000;
    NumMaxCiclosSinMejorarValidacion = 100;

    numEjecuciones = 10;
    numPatrones = size(patrones,2);

    precisionesEntrenamiento = Array{Float64,1}();
    precisionesValidacion = Array{Float64,1}();
    precisionesTest = Array{Float64,1}();
    
    

    @assert (size(patrones,2)==size(salidasDeseadas,2));
    # Si vamos a separar dos clases, solamente necesitamos una salida
    #  Por tanto, nos aseguramos de que no haya 2 salidas
    @assert (numSalidasRNA!=2);
    
    mejorModelo = Chain(
        Dense(numEntradasRNA, ArquitecturaRNA[1], σ),
        Dense(ArquitecturaRNA[1], numSalidasRNA, σ), );
    
    mejorCapa1 = 1
    mejorCapa2 = 1
    
    for capa1 in 1:5
        for capa2 in 1:5
            
            println("Arquitectura : 6:",capa1,":",capa2,":1")
            
            for numEjecucion in 1:numEjecuciones
                print("Ejecucion ", numEjecucion)
                (indicesPatronesEntrenamiento, indicesPatronesValidacion, indicesPatronesTest) = 
                    holdOut(numPatrones, PorcentajeValidacion, PorcentajeTest);

                funcionTransferenciaCapasOcultas = σ; 

                if(length(ArquitecturaRNA) == 1) 
                    modelo = Chain(
                        Dense(numEntradasRNA,     ArquitecturaRNA[1], funcionTransferenciaCapasOcultas),
                        Dense(ArquitecturaRNA[1], numSalidasRNA,      σ                               ), );
                elseif(length(ArquitecturaRNA)==2)
                    modelo = Chain(
                        Dense(numEntradasRNA,     capa1, funcionTransferenciaCapasOcultas),
                        Dense(capa1, capa2, funcionTransferenciaCapasOcultas),
                        Dense(capa2, numSalidasRNA,      σ                               ), );
                else 
                    error("Para redes MLP, no hacer más de 2 capas ocultas")
                end
                println(modelo)

                loss(x, y) = (size(y,1)>1) ? 
                    crossentropy(modelo(x), y) : 
                    mean(binarycrossentropy.(modelo(x), y));

                # Entrenamos la RNA hasta que se cumpla el criterio de parada
                criterioFin = false;
                numCiclo = 0; mejorLossValidacion = Inf; numCiclosSinMejorarValidacion = 0; mejorModelo = nothing;
                while (!criterioFin)

                    Flux.train!(loss, params(modelo), [(patrones[:,indicesPatronesEntrenamiento], salidasDeseadas[:,indicesPatronesEntrenamiento])], ADAM(0.01));

                    numCiclo += 1;

                    if (PorcentajeValidacion>0)
                        lossValidacion = loss(patrones[:,indicesPatronesValidacion], salidasDeseadas[:,indicesPatronesValidacion]);
                        if (lossValidacion<mejorLossValidacion)
                            mejorLossValidacion = lossValidacion;
                            mejorModelo = deepcopy(modelo);
                            mejorCapa1 = capa1
                            mejorCapa2 = capa2
                            numCiclosSinMejorarValidacion = 0;
                        else
                            numCiclosSinMejorarValidacion += 1;
                        end;
                    end;

                    if (numCiclo>=NumMaxCiclosEntrenamiento)
                        criterioFin = true;
                    end;
                    if (numCiclosSinMejorarValidacion>NumMaxCiclosSinMejorarValidacion)
                        criterioFin = true;
                    end;

                end;

                if (PorcentajeValidacion>0)
                    modelo = mejorModelo;
                end;
                # Fin del entrenamiento de la RNA

                println("   RNA entrenada durante ", numCiclo, " ciclos");
                 precision(x, y) = (size(y,1)==1) ? 
                    mean((modelo(x).>=0.5) .== y) : 
                    mean(onecold(modelo(x)) .== onecold(y));

                precisionEntrenamiento = 100*precision(patrones[:,indicesPatronesEntrenamiento], salidasDeseadas[:,indicesPatronesEntrenamiento]);
                println("   Precision en el conjunto de entrenamiento: $precisionEntrenamiento %");
                push!(precisionesEntrenamiento, precisionEntrenamiento);

                if (PorcentajeValidacion>0)
                    precisionValidacion = 100*precision(patrones[:,indicesPatronesValidacion], salidasDeseadas[:,indicesPatronesValidacion]);
                    println("   Precision en el conjunto de validacion: $precisionValidacion %");
                    push!(precisionesValidacion,    precisionValidacion);
                end;

                precisionTest = 100*precision(patrones[:,indicesPatronesTest], salidasDeseadas[:,indicesPatronesTest]);
                println("   Precision en el conjunto de test: $precisionTest %");
                push!(precisionesTest,          precisionTest);

            end;

            println("Resultados en promedio:");
            println("Entrenamiento: ", mean(precisionesEntrenamiento), " %, desviacion tipica: ", std(precisionesEntrenamiento));
            if (PorcentajeValidacion>0)
                println("Validacion: ", mean(precisionesValidacion)," %, desviacion tipica: ", std(precisionesValidacion));
            end;
            println("Test: ", mean(precisionesTest), " %, desviacion tipica: ", std(precisionesTest));
        end
    end
    return mejorModelo, mejorCapa1, mejorCapa2;
end;

In [5]:
function find_nemo(mejorModelo, file, n_inputs, wall)
    imagen = load(file)
    xtrain = zeros(1,n_inputs)
    
    reps = 0
    num_reps = 71
    count = 0
    height = 50
    width=100

    while reps < num_reps  
        i = rand(1:height);
        j = rand(1:width);
        while i < (size(imagen,1) - height)
            j = rand(1:width);
            while j < (size(imagen,2) - width)
    
                img_v = @view imagen[i:i+height, j:j+width]
                
                xtrain = process_image(img_v, n_inputs);
                
                if((reps % 70 == 0) && count == 0)
                    wall -= 0.0005
                elseif (reps % 10 == 0) 
                        reps = num_reps
                end
    
                if(mejorModelo(xtrain)[1,1] > wall)
                    count+=1;
                    #println(mejorModelo(reshape(xtrain,6,1))[1,1])
                    imagen[i:i+height, j] .= RGB(0,1,0);
                    imagen[i:i+height, j+width] .= RGB(0,1,0);
                    imagen[i, j:j+width] .= RGB(0,1,0);
                    imagen[i+height, j:j+width] .= RGB(0,1,0);
                end
                j+=width;
            end
            i+=height;
        end
        reps+=1;
        height = rand(20:floor(Int, size(imagen)[1]/3))
        width = rand(40:floor(Int, size(imagen)[2]/3))
    end
    
    println(wall)
    #imshow(imagen)
    display(imagen);
end;

In [6]:
patrones_si, salidas_deseadas_si = prepare_data("positivos", 6, 1);
patrones_no, salidas_deseadas_no = prepare_data("negativos", 6, 0);

In [7]:
patrones = hcat(patrones_si, patrones_no);
salidasDeseadas = hcat(salidas_deseadas_si, salidas_deseadas_no);

In [40]:
mejorModelo, mejorC1, mejorC2 = train_model(6,1,[5, 4], patrones, salidasDeseadas);

println("Mejor Arquitectura -> 6:",mejorC1,":",mejorC2,":1")

Arquitectura : 6:1:1:1
Ejecucion 1Chain(Dense(6, 1, σ), Dense(1, 1, σ), Dense(1, 1, σ))
   RNA entrenada durante 1000 ciclos
   Precision en el conjunto de entrenamiento: 80.73394495412845 %
   Precision en el conjunto de validacion: 77.77777777777779 %
   Precision en el conjunto de test: 91.66666666666666 %
Ejecucion 2Chain(Dense(6, 1, σ), Dense(1, 1, σ), Dense(1, 1, σ))
   RNA entrenada durante 1000 ciclos
   Precision en el conjunto de entrenamiento: 84.40366972477065 %
   Precision en el conjunto de validacion: 86.11111111111111 %
   Precision en el conjunto de test: 88.88888888888889 %
Ejecucion 3Chain(Dense(6, 1, σ), Dense(1, 1, σ), Dense(1, 1, σ))
   RNA entrenada durante 1000 ciclos
   Precision en el conjunto de entrenamiento: 85.3211009174312 %
   Precision en el conjunto de validacion: 80.55555555555556 %
   Precision en el conjunto de test: 80.55555555555556 %
Ejecucion 4Chain(Dense(6, 1, σ), Dense(1, 1, σ), Dense(1, 1, σ))
   RNA entrenada durante 102 ciclos
   Precision 

   RNA entrenada durante 770 ciclos
   Precision en el conjunto de entrenamiento: 86.23853211009175 %
   Precision en el conjunto de validacion: 77.77777777777779 %
   Precision en el conjunto de test: 86.11111111111111 %
Ejecucion 9Chain(Dense(6, 1, σ), Dense(1, 3, σ), Dense(3, 1, σ))
   RNA entrenada durante 775 ciclos
   Precision en el conjunto de entrenamiento: 84.40366972477065 %
   Precision en el conjunto de validacion: 80.55555555555556 %
   Precision en el conjunto de test: 83.33333333333334 %
Ejecucion 10Chain(Dense(6, 1, σ), Dense(1, 3, σ), Dense(3, 1, σ))
   RNA entrenada durante 1000 ciclos
   Precision en el conjunto de entrenamiento: 83.4862385321101 %
   Precision en el conjunto de validacion: 86.11111111111111 %
   Precision en el conjunto de test: 75.0 %
Resultados en promedio:
Entrenamiento: 75.81039755351684 %, desviacion tipica: 14.614464981829903
Validacion: 74.25925925925927 %, desviacion tipica: 14.162690699916501
Test: 73.7037037037037 %, desviacion tipica: 15

   RNA entrenada durante 1000 ciclos
   Precision en el conjunto de entrenamiento: 82.56880733944955 %
   Precision en el conjunto de validacion: 88.88888888888889 %
   Precision en el conjunto de test: 88.88888888888889 %
Ejecucion 2Chain(Dense(6, 2, σ), Dense(2, 4, σ), Dense(4, 1, σ))
   RNA entrenada durante 680 ciclos
   Precision en el conjunto de entrenamiento: 83.4862385321101 %
   Precision en el conjunto de validacion: 83.33333333333334 %
   Precision en el conjunto de test: 86.11111111111111 %
Ejecucion 3Chain(Dense(6, 2, σ), Dense(2, 4, σ), Dense(4, 1, σ))
   RNA entrenada durante 298 ciclos
   Precision en el conjunto de entrenamiento: 77.06422018348624 %
   Precision en el conjunto de validacion: 63.888888888888886 %
   Precision en el conjunto de test: 91.66666666666666 %
Ejecucion 4Chain(Dense(6, 2, σ), Dense(2, 4, σ), Dense(4, 1, σ))
   RNA entrenada durante 295 ciclos
   Precision en el conjunto de entrenamiento: 75.22935779816514 %
   Precision en el conjunto de valid

   RNA entrenada durante 516 ciclos
   Precision en el conjunto de entrenamiento: 77.06422018348624 %
   Precision en el conjunto de validacion: 88.88888888888889 %
   Precision en el conjunto de test: 83.33333333333334 %
Ejecucion 9Chain(Dense(6, 3, σ), Dense(3, 1, σ), Dense(1, 1, σ))
   RNA entrenada durante 117 ciclos
   Precision en el conjunto de entrenamiento: 41.284403669724774 %
   Precision en el conjunto de validacion: 50.0 %
   Precision en el conjunto de test: 22.22222222222222 %
Ejecucion 10Chain(Dense(6, 3, σ), Dense(3, 1, σ), Dense(1, 1, σ))
   RNA entrenada durante 1000 ciclos
   Precision en el conjunto de entrenamiento: 86.23853211009175 %
   Precision en el conjunto de validacion: 83.33333333333334 %
   Precision en el conjunto de test: 77.77777777777779 %
Resultados en promedio:
Entrenamiento: 77.16430358632194 %, desviacion tipica: 14.175433688863011
Validacion: 76.28787878787878 %, desviacion tipica: 12.173224885480455
Test: 75.2020202020202 %, desviacion tipica: 

   RNA entrenada durante 630 ciclos
   Precision en el conjunto de entrenamiento: 80.73394495412845 %
   Precision en el conjunto de validacion: 91.66666666666666 %
   Precision en el conjunto de test: 86.11111111111111 %
Ejecucion 5Chain(Dense(6, 3, σ), Dense(3, 4, σ), Dense(4, 1, σ))
   RNA entrenada durante 1000 ciclos
   Precision en el conjunto de entrenamiento: 82.56880733944955 %
   Precision en el conjunto de validacion: 83.33333333333334 %
   Precision en el conjunto de test: 77.77777777777779 %
Ejecucion 6Chain(Dense(6, 3, σ), Dense(3, 4, σ), Dense(4, 1, σ))
   RNA entrenada durante 520 ciclos
   Precision en el conjunto de entrenamiento: 80.73394495412845 %
   Precision en el conjunto de validacion: 83.33333333333334 %
   Precision en el conjunto de test: 86.11111111111111 %
Ejecucion 7Chain(Dense(6, 3, σ), Dense(3, 4, σ), Dense(4, 1, σ))
   RNA entrenada durante 1000 ciclos
   Precision en el conjunto de entrenamiento: 83.4862385321101 %
   Precision en el conjunto de valid

   RNA entrenada durante 1000 ciclos
   Precision en el conjunto de entrenamiento: 78.89908256880734 %
   Precision en el conjunto de validacion: 86.11111111111111 %
   Precision en el conjunto de test: 83.33333333333334 %
Ejecucion 2Chain(Dense(6, 4, σ), Dense(4, 2, σ), Dense(2, 1, σ))
   RNA entrenada durante 286 ciclos
   Precision en el conjunto de entrenamiento: 79.81651376146789 %
   Precision en el conjunto de validacion: 61.111111111111114 %
   Precision en el conjunto de test: 69.44444444444444 %
Ejecucion 3Chain(Dense(6, 4, σ), Dense(4, 2, σ), Dense(2, 1, σ))
   RNA entrenada durante 594 ciclos
   Precision en el conjunto de entrenamiento: 82.56880733944955 %
   Precision en el conjunto de validacion: 88.88888888888889 %
   Precision en el conjunto de test: 69.44444444444444 %
Ejecucion 4Chain(Dense(6, 4, σ), Dense(4, 2, σ), Dense(2, 1, σ))
   RNA entrenada durante 571 ciclos
   Precision en el conjunto de entrenamiento: 86.23853211009175 %
   Precision en el conjunto de vali

   RNA entrenada durante 1000 ciclos
   Precision en el conjunto de entrenamiento: 82.56880733944955 %
   Precision en el conjunto de validacion: 80.55555555555556 %
   Precision en el conjunto de test: 80.55555555555556 %
Ejecucion 9Chain(Dense(6, 4, σ), Dense(4, 4, σ), Dense(4, 1, σ))
   RNA entrenada durante 1000 ciclos
   Precision en el conjunto de entrenamiento: 84.40366972477065 %
   Precision en el conjunto de validacion: 72.22222222222221 %
   Precision en el conjunto de test: 72.22222222222221 %
Ejecucion 10Chain(Dense(6, 4, σ), Dense(4, 4, σ), Dense(4, 1, σ))
   RNA entrenada durante 608 ciclos
   Precision en el conjunto de entrenamiento: 82.56880733944955 %
   Precision en el conjunto de validacion: 75.0 %
   Precision en el conjunto de test: 86.11111111111111 %
Resultados en promedio:
Entrenamiento: 79.00048285852246 %, desviacion tipica: 11.927665702368607
Validacion: 77.66081871345028 %, desviacion tipica: 10.789857511940449
Test: 77.5 %, desviacion tipica: 13.660825955

   RNA entrenada durante 1000 ciclos
   Precision en el conjunto de entrenamiento: 82.56880733944955 %
   Precision en el conjunto de validacion: 83.33333333333334 %
   Precision en el conjunto de test: 77.77777777777779 %
Ejecucion 6Chain(Dense(6, 5, σ), Dense(5, 2, σ), Dense(2, 1, σ))
   RNA entrenada durante 463 ciclos
   Precision en el conjunto de entrenamiento: 82.56880733944955 %
   Precision en el conjunto de validacion: 88.88888888888889 %
   Precision en el conjunto de test: 72.22222222222221 %
Ejecucion 7Chain(Dense(6, 5, σ), Dense(5, 2, σ), Dense(2, 1, σ))
   RNA entrenada durante 1000 ciclos
   Precision en el conjunto de entrenamiento: 85.3211009174312 %
   Precision en el conjunto de validacion: 75.0 %
   Precision en el conjunto de test: 77.77777777777779 %
Ejecucion 8Chain(Dense(6, 5, σ), Dense(5, 2, σ), Dense(2, 1, σ))
   RNA entrenada durante 518 ciclos
   Precision en el conjunto de entrenamiento: 84.40366972477065 %
   Precision en el conjunto de validacion: 91.666

   RNA entrenada durante 1000 ciclos
   Precision en el conjunto de entrenamiento: 80.73394495412845 %
   Precision en el conjunto de validacion: 83.33333333333334 %
   Precision en el conjunto de test: 86.11111111111111 %
Ejecucion 3Chain(Dense(6, 5, σ), Dense(5, 5, σ), Dense(5, 1, σ))
   RNA entrenada durante 1000 ciclos
   Precision en el conjunto de entrenamiento: 85.3211009174312 %
   Precision en el conjunto de validacion: 83.33333333333334 %
   Precision en el conjunto de test: 75.0 %
Ejecucion 4Chain(Dense(6, 5, σ), Dense(5, 5, σ), Dense(5, 1, σ))
   RNA entrenada durante 1000 ciclos
   Precision en el conjunto de entrenamiento: 79.81651376146789 %
   Precision en el conjunto de validacion: 80.55555555555556 %
   Precision en el conjunto de test: 80.55555555555556 %
Ejecucion 5Chain(Dense(6, 5, σ), Dense(5, 5, σ), Dense(5, 1, σ))
   RNA entrenada durante 1000 ciclos
   Precision en el conjunto de entrenamiento: 83.4862385321101 %
   Precision en el conjunto de validacion: 80.55

In [ ]:
directorio = readdir("fotos");

for file in directorio 
    println(file);
    find_nemo(mejorModelo, string("fotos/",file), 6, 0.85);
end;


In [ ]:
function get_orange_white_boolean(file)
    imagen = load(file)
    diferenciaRojoVerde = 0.3; diferenciaRojoAzul = 0.3;
    canalRojo = red.(imagen); 
    canalVerde = green.(imagen); 
    canalAzul = blue.(imagen);
    white_chanel = (zeros(size(canalRojo)[1], size(canalRojo)[2]).+0.6);
    black_chanel = (zeros(size(canalRojo)[1], size(canalRojo)[2]).+0.1);
    
     #=
    mb_w =  (canalRojo.>white_chanel) .& 
            (canalVerde.>white_chanel) .& 
            (canalAzul.>white_chanel);
   
    mb_b =  (canalRojo.<black_chanel) .& 
            (canalVerde.<black_chanel) .& 
            (canalAzul.<black_chanel); =#
    
    mb_r =  (canalRojo.>(canalVerde.+diferenciaRojoVerde)) .& 
            (canalRojo.>(canalAzul.+diferenciaRojoAzul));
    
    matrizBooleana = mb_r; 
    
    return matrizBooleana;
end;

In [ ]:
file_path = "fotos/pez002.jpg";
labelArray = ImageMorphology.label_components(get_orange_white_boolean(file_path));

In [ ]:

boundingBoxes = ImageMorphology.component_boxes(labelArray);
tamanos = ImageMorphology.component_lengths(labelArray); #Tamaño del grupo 
pixeles = ImageMorphology.component_indices(labelArray);
pixeles = ImageMorphology.component_subscripts(labelArray);
centroides = ImageMorphology.component_centroids(labelArray);

tamanos = component_lengths(labelArray);
etiquetasEliminar = findall(tamanos .<= 370) .- 1;
matrizBooleana = [!in(etiqueta,etiquetasEliminar) && (etiqueta!=0) for etiqueta in labelArray];
display(Gray.(matrizBooleana));

In [ ]:
labelArray = ImageMorphology.label_components(matrizBooleana);
imagenObjetos = RGB.(matrizBooleana, matrizBooleana, matrizBooleana);
centroides = ImageMorphology.component_centroids(labelArray)[2:end];
for centroide in centroides
    x = Int(round(centroide[1]));
    y = Int(round(centroide[2]));
    imagenObjetos[ x, y ] = RGB(1,0,0);
end;
# Vamos a recuadrar el bounding box de estos objetos, en color verde
# Calculamos los bounding boxes, y eliminamos el primero (el objeto "0")
boundingBoxes = ImageMorphology.component_boxes(labelArray)[2:end];
for boundingBox in boundingBoxes
    x1 = boundingBox[1][1];
    y1 = boundingBox[1][2];
    x2 = boundingBox[2][1];
    y2 = boundingBox[2][2];
    imagenObjetos[ x1:x2 , y1 ] .= RGB(0,1,0);
    imagenObjetos[ x1:x2 , y2 ] .= RGB(0,1,0);
    imagenObjetos[ x1 , y1:y2 ] .= RGB(0,1,0);
    imagenObjetos[ x2 , y1:y2 ] .= RGB(0,1,0);
end;
display(imagenObjetos);

In [ ]:
file_path = "fotos/pez002.jpg"
imagen = load(file_path)

In [ ]:
# Y hacemos lo mismo con la imagen original:
for boundingBox in boundingBoxes
    x1 = boundingBox[1][1];
    y1 = boundingBox[1][2];
    x2 = boundingBox[2][1];
    y2 = boundingBox[2][2];
    imagen[ x1:x2 , y1 ] .= RGB(0,1,0);
    imagen[ x1:x2 , y2 ] .= RGB(0,1,0);
    imagen[ x1 , y1:y2 ] .= RGB(0,1,0);
    imagen[ x2 , y1:y2 ] .= RGB(0,1,0);
end;
display(imagen);

In [ ]:
function get_orange_white_boolean(file)
    imagen = load(file)
    diferenciaRojoVerde = 0.3; diferenciaRojoAzul = 0.3;
    canalRojo = red.(imagen); 
    canalVerde = green.(imagen); 
    canalAzul = blue.(imagen);
    white_chanel = (zeros(size(canalRojo)[1], size(canalRojo)[2]).+0.6);
    black_chanel = (zeros(size(canalRojo)[1], size(canalRojo)[2]).+0.1);
    
     #=
    mb_w =  (canalRojo.>white_chanel) .& 
            (canalVerde.>white_chanel) .& 
            (canalAzul.>white_chanel);
   
    mb_b =  (canalRojo.<black_chanel) .& 
            (canalVerde.<black_chanel) .& 
            (canalAzul.<black_chanel); =#
    
    mb_r =  (canalRojo.>(canalVerde.+diferenciaRojoVerde)) .& 
            (canalRojo.>(canalAzul.+diferenciaRojoAzul));
    
    matrizBooleana = mb_r; 
    
    return matrizBooleana;
end;

In [ ]:
function check_region(x1, y1, x2, y2, imagen, modelo, nImputs, wall)
    img_v = @view imagen[x1:x2, y1:y2]
    img_CHW = channelview(img_v)

    xtrain = process_image(img_v, nImputs);
    
    if(modelo(reshape(xtrain,nImputs,1))[1,1] > wall)
        imagen[x1:x2, y1] .= RGB(0,1,0);
        imagen[x1:x2, y2] .= RGB(0,1,0);
        imagen[x1, y1:y2] .= RGB(0,1,0);
        imagen[x2, y1:y2] .= RGB(0,1,0);
    else
        imagen[x1:x2, y1] .= RGB(1,0,0);
        imagen[x1:x2, y2] .= RGB(1,0,0);
        imagen[x1, y1:y2] .= RGB(1,0,0);
        imagen[x2, y1:y2] .= RGB(1,0,0);
    end
end

In [ ]:
function process_picture(filePath, modelo, nImputs, wall)
    booleanMatrix = dilate(dilate(dilate(get_orange_white_boolean(filePath))));
    labelArray = ImageMorphology.label_components(booleanMatrix);
    boundingBoxes = ImageMorphology.component_boxes(labelArray);
    tamanos = ImageMorphology.component_lengths(labelArray); #Tamaño del grupo 
    pixeles = ImageMorphology.component_indices(labelArray);
    pixeles = ImageMorphology.component_subscripts(labelArray);
    centroides = ImageMorphology.component_centroids(labelArray);

    tamanos = component_lengths(labelArray);
    etiquetasEliminar = findall(tamanos .<= 370) .- 1;
    matrizBooleana = [!in(etiqueta,etiquetasEliminar) && (etiqueta!=0) for etiqueta in labelArray];
    
    labelArray = ImageMorphology.label_components(matrizBooleana);
    imagenObjetos = RGB.(matrizBooleana, matrizBooleana, matrizBooleana);
    centroides = ImageMorphology.component_centroids(labelArray)[2:end];
    for centroide in centroides
        x = Int(round(centroide[1]));
        y = Int(round(centroide[2]));
        imagenObjetos[ x, y ] = RGB(1,0,0);
    end;
    # Vamos a recuadrar el bounding box de estos objetos, en color verde
    # Calculamos los bounding boxes, y eliminamos el primero (el objeto "0")
    imagen = load(filePath)
    
    boundingBoxes = ImageMorphology.component_boxes(labelArray)[2:end];
    for boundingBox in boundingBoxes
        x1 = boundingBox[1][1];
        y1 = boundingBox[1][2];
        x2 = boundingBox[2][1];
        y2 = boundingBox[2][2];

        check_region(x1,y1,x2,y2, imagen, modelo, nImputs, wall)
    end;
    display(imagen);
end;

In [ ]:
process_picture("fotos/pez004.jpg", mejorModelo, 6, 0.72)

In [ ]:
directorio = readdir("fotos");

for file in directorio 
    println(file);
    process_picture(string("fotos/",file), mejorModelo, 6, 0.72);
end;

In [ ]:
using ImageView, Images, ImageSegmentation

segment_to_image(segments) = map(i->segment_mean(segments, i), labels_map(segments));

function background_object_filter(file)
    img = load(file);
    # find segments
    segments_1 = fast_scanning(img, 0.05)
    segments_2 = fast_scanning(img, 0.15)
    segments_3 = fast_scanning(img, 0.2)

    # define helper values
    color_black = RGB4(0.,0.,0.)
    img_width = size(img, 2)

    # create a new image which will be a stack
    # of 3 different results of felzenszwalb algorithm
    new_img = fill(color_black, size(img) .* (1, 3))
    new_img[:, 1:img_width] .= segment_to_image(segments_1)
    new_img[:, img_width+1:img_width*2] .= segment_to_image(segments_2)
    new_img[:, img_width*2+1:img_width*3] .= segment_to_image(segments_3)
    new_img[:, img_width] = new_img[:, img_width*2] .= RGB4(0.,0.,0.)

    deletion_rule = i -> (segment_pixel_count(segments_2,i) < 750)
    replacement_rule = (i,j) -> (-segment_pixel_count(segments_2, j))
    segments_n = prune_segments(segments_2, deletion_rule, replacement_rule)

    # preview the result

    return segment_to_image(segments_n)
end;

In [ ]:


# load an image and create a grayscale copy
img = load("fotos/pez001.jpg")
img_gray = RGB.(Gray.(img))

# get channels representation
img_channel_view = channelview(img);
img_gray_channel_view = channelview(img_gray);

# make channel dimension last and crop the required are
img_arr = permuteddimsview(img_channel_view, (2, 3, 1));
img_gray_arr = permuteddimsview(img_gray_channel_view, (2, 3, 1));

# create a mask with all values being true
img_mask = fill(true, size(img));

# spot are with colors to retain
img_spot_height = 430:460
img_spot_width = 430:460

for channel_id = 1:3

    # select current channel and crop ares of interest
    current_channel = view(img_arr, :, :, channel_id)
    current_channel_area = current_channel[img_spot_height, img_spot_width, :]

    # identify min and max values in a cropped area
    channel_min = minimum(current_channel_area)
    channel_max = maximum(current_channel_area)

    # merge existing mask with a channel specific mask
    channel_mask = channel_min .< current_channel .< channel_max
    img_mask = img_mask .& channel_mask
end

# apply mask
img_masked = img_arr .* img_mask .+ img_gray_arr .* .~(img_mask);
final_image = colorview(RGB, permutedims(img_masked, (3, 1, 2)))
final_image

In [ ]:
using Images, ImageView, ImageSegmentation
img = load("fotos/pez001.jpg");

seeds = [
    (CartesianIndex(220,250), 1), # object location
    (CartesianIndex(220,500), 2)  # background location
]

segments = seeded_region_growing(img, seeds)

map(i->segment_mean(segments,i), labels_map(segments))

In [ ]:
img = load("fotos/pez001.jpg");
seeds = [
    (CartesianIndex(240,120), 1), 
    (CartesianIndex(295,70), 2),
    (CartesianIndex(319,40), 3),
    (CartesianIndex(90,300), 4),
    (CartesianIndex(295,325), 5),
    (CartesianIndex(76,135), 6)
]

segments = seeded_region_growing(img, seeds)
map(i->segment_mean(segments,i), labels_map(segments))

In [ ]:
# find segments
img = load("fotos/pez001.jpg");

segments_1 = felzenszwalb(img, 75)
segments_2 = felzenszwalb(img, 75, 150)
segments_3 = felzenszwalb(img, 75, 350)

img_width = size(img, 2)

new_img = hcat(
    segment_to_image(segments_1),
    segment_to_image(segments_2),
    segment_to_image(segments_3)
)
new_img[:, img_width] .= new_img[:, img_width*2] .= colorant"black"

new_img




In [ ]:
img = load("fotos/pez003.jpg");
segments_1 = felzenszwalb(img, 75, 350)
segment_to_image(segments_1)



In [ ]:
function get_orange_white_boolean(imagen)
    
    diferenciaRojoVerde = 0.1; diferenciaRojoAzul = 0.15;
    canalRojo = red.(imagen); 
    canalVerde = green.(imagen); 
    canalAzul = blue.(imagen);
    white_chanel = (zeros(size(canalRojo)[1], size(canalRojo)[2]).+0.80);
    white_chanel1 = (zeros(size(canalRojo)[1], size(canalRojo)[2]).+0.90);
    
    mb_w =  (canalRojo.>white_chanel.+ mean(canalRojo)*0.5) .& 
            (canalVerde.>white_chanel.- mean(canalVerde)) .& 
            (canalAzul.>white_chanel.- mean(canalAzul));
    
    mb_w1 =  (canalRojo.>white_chanel1) .& 
            (canalVerde.>white_chanel1) .& 
            (canalAzul.>white_chanel1);
    
    #=

    mb_b =  (canalRojo.<black_chanel) .& 
            (canalVerde.<black_chanel) .& 
            (canalAzul.<black_chanel); =#
    
    mb_r =  (canalRojo.>(canalVerde.+diferenciaRojoVerde)) .& 
            (canalRojo.>(canalAzul.+diferenciaRojoAzul));
    
    matrizBooleana = mb_r .| mb_w .| mb_w1; 
    
    return matrizBooleana;
end;

In [ ]:
img = load("fotos/pez026.jpg");
display(img);

In [ ]:
#segments_1 = felzenszwalb(img, 75, 350)
segments_1 = felzenszwalb(img, 4)
display(segment_to_image(segments_1))

In [ ]:

matrizBooleana = get_orange_white_boolean(segment_to_image(segments_1))
#matrizBooleana = get_orange_white_boolean(img)
display(Gray.(matrizBooleana));


In [ ]:
img = load("fotos/pez010.jpg");
segments_1 = felzenszwalb(img,4)
matrizBooleana = get_orange_white_boolean(segment_to_image(segments_1))
#matrizBooleana = get_orange_white_boolean(img)
labelArray = ImageMorphology.label_components(matrizBooleana);

boundingBoxes = ImageMorphology.component_boxes(labelArray);
tamanos = ImageMorphology.component_lengths(labelArray); #Tamaño del grupo 
pixeles = ImageMorphology.component_indices(labelArray);
pixeles = ImageMorphology.component_subscripts(labelArray);
centroides = ImageMorphology.component_centroids(labelArray);

tamanos = component_lengths(labelArray);

min_size = (size(img)[1]*size(img)[2]/600);
println(min_size);
etiquetasEliminar = findall(tamanos .<= min_size) .- 1;
matrizBooleana = [!in(etiqueta,etiquetasEliminar) && (etiqueta!=0) for etiqueta in labelArray];
display(Gray.(matrizBooleana));

In [ ]:
function process_image(image, n_inputs)
    img_CHW = channelview(image);
    patron = zeros(n_inputs);
    
    patron[1] = mean(img_CHW[1,:,:]);
    patron[2] = mean(img_CHW[2,:,:]);
    patron[3] = mean(img_CHW[3,:,:]);
    patron[4] = std(img_CHW[1,:,:]);
    patron[5] = std(img_CHW[2,:,:]);
    patron[6] = std(img_CHW[3,:,:]);

    return reshape(patron,n_inputs,1);
end;

In [ ]:
function get_orange_white_boolean(imagen)
    
    diferenciaRojoVerde = 0.3; 
    diferenciaRojoAzul = 0.3;
    
    canalRojo = red.(imagen); 
    canalVerde = green.(imagen); 
    canalAzul = blue.(imagen);
    
    white_chanel = (zeros(size(canalRojo)[1], size(canalRojo)[2]).+0.6);
    white_chanel1 = (zeros(size(canalRojo)[1], size(canalRojo)[2]).+0.9);
    
    mb_w =  (canalRojo.>white_chanel.+ mean(canalRojo)) .& 
            (canalVerde.>white_chanel.- mean(canalVerde)) .& 
            (canalAzul.>white_chanel.- mean(canalAzul));
    
    mb_w1 =  (canalRojo.>white_chanel1) .& 
            (canalVerde.>white_chanel1) .& 
            (canalAzul.>white_chanel1);
    
    mb_r =  (canalRojo.>(canalVerde.+diferenciaRojoVerde)) .& 
            (canalRojo.>(canalAzul.+diferenciaRojoAzul));
    
    matrizBooleana = mb_r .| mb_w1 #.| mb_w1; 
    
    return matrizBooleana;
end;

In [ ]:


function check_region(x1, y1, x2, y2, imagen, modelo, nImputs, wall)
    img_v = @view imagen[x1:x2, y1:y2]
    #img_CHW = channelview(img_v)

    xtrain = process_image(img_v, nImputs);
    
    if(modelo(reshape(xtrain,nImputs,1))[1,1] > wall)
        imagen[x1:x2, y1] .= RGB(0,1,0);
        imagen[x1:x2, y2] .= RGB(0,1,0);
        imagen[x1, y1:y2] .= RGB(0,1,0);
        imagen[x2, y1:y2] .= RGB(0,1,0);
    else
        imagen[x1:x2, y1] .= RGB(1,0,0);
        imagen[x1:x2, y2] .= RGB(1,0,0);
        imagen[x1, y1:y2] .= RGB(1,0,0);
        imagen[x2, y1:y2] .= RGB(1,0,0);
    end
end;

function process_picture(filePath, modelo, nImputs, wall)
    imagen = load(filePath);
    #segments_1 = felzenszwalb(imagen, 4);
    #booleanMatrix = get_orange_white_boolean(segment_to_image(segments_1));
    booleanMatrix = get_orange_white_boolean(imagen)
    booleanMatrix = closing(dilate(booleanMatrix,[1,2]))

    
    if(!maximum(booleanMatrix))
        booleanMatrix = get_orange_white_boolean(imagen);
    end;
    
    labelArray = ImageMorphology.label_components(booleanMatrix);
    boundingBoxes = ImageMorphology.component_boxes(labelArray);
    tamanos = ImageMorphology.component_lengths(labelArray); #Tamaño del grupo 
    pixeles = ImageMorphology.component_indices(labelArray);
    pixeles = ImageMorphology.component_subscripts(labelArray);
    centroides = ImageMorphology.component_centroids(labelArray);

    tamanos = component_lengths(labelArray);
    min_size = (size(img)[1]*size(img)[2]/600);
    etiquetasEliminar = findall(tamanos .<= min_size) .- 1;
    matrizBooleana = [!in(etiqueta,etiquetasEliminar) && (etiqueta!=0) for etiqueta in labelArray];
    
    labelArray = ImageMorphology.label_components(matrizBooleana);
    imagenObjetos = RGB.(matrizBooleana, matrizBooleana, matrizBooleana);
    centroides = ImageMorphology.component_centroids(labelArray)[2:end];
    
    for centroide in centroides
        x = Int(round(centroide[1]));
        y = Int(round(centroide[2]));
        imagenObjetos[ x, y ] = RGB(1,0,0);
    end;
    
    
   
    imagen = load(filePath)
    
    boundingBoxes = ImageMorphology.component_boxes(labelArray)[2:end];
    
    for boundingBox in boundingBoxes
        x1 = boundingBox[1][1];
        y1 = boundingBox[1][2];
        x2 = boundingBox[2][1];
        y2 = boundingBox[2][2];

        check_region(x1,y1,x2,y2, imagen, modelo, nImputs, wall)
    end;
    return imagen;
end;

In [ ]:
directorio = readdir("fotos");

for file in directorio 
    println(file);
    processed = process_picture(string("fotos/",file), mejorModelo, 6, 0.72);
    
    
    display(processed);
end;

#=
processed = process_picture("fotos/pez006.jpg", mejorModelo, 6, 0.72);
display(processed);=#

In [ ]:

img = load("fotos/pez001.jpg");

# find segments
segments_1 = fast_scanning(img, 0.05)
segments_2 = fast_scanning(img, 0.15)
segments_3 = fast_scanning(img, 0.2)

# define helper values
color_black = RGB4(0.,0.,0.)
img_width = size(img, 2)

# create a new image which will be a stack
# of 3 different results of felzenszwalb algorithm
new_img = fill(color_black, size(img) .* (1, 3))
new_img[:, 1:img_width] .= segment_to_image(segments_1)
new_img[:, img_width+1:img_width*2] .= segment_to_image(segments_2)
new_img[:, img_width*2+1:img_width*3] .= segment_to_image(segments_3)
new_img[:, img_width] = new_img[:, img_width*2] .= RGB4(0.,0.,0.)

deletion_rule = i -> (segment_pixel_count(segments_2,i) < 750)
replacement_rule = (i,j) -> (-segment_pixel_count(segments_2, j))
segments_n = prune_segments(segments_2, deletion_rule, replacement_rule)

# preview the result

segment_to_image(segments_n)



In [ ]:
function test()

    img = load("fotos/pez006.jpg");

    # find segments
    segments_1 = fast_scanning(img, 0.05)
    segments_2 = fast_scanning(img, 0.15)
    segments_3 = fast_scanning(img, 0.2)

    # define helper values
    color_black = RGB4(0.,0.,0.)
    img_width = size(img, 2)

    # create a new image which will be a stack
    # of 3 different results of felzenszwalb algorithm
    new_img = fill(color_black, size(img) .* (1, 3))
    new_img[:, 1:img_width] .= segment_to_image(segments_1)
    new_img[:, img_width+1:img_width*2] .= segment_to_image(segments_2)
    new_img[:, img_width*2+1:img_width*3] .= segment_to_image(segments_3)
    new_img[:, img_width] = new_img[:, img_width*2] .= RGB4(0.,0.,0.)

    deletion_rule = i -> (segment_pixel_count(segments_2,i) < 750)
    replacement_rule = (i,j) -> (-segment_pixel_count(segments_2, j))
    segments_n = prune_segments(segments_2, deletion_rule, replacement_rule)

    # preview the result

    segment_to_image(segments_n)
    end;

test()


In [ ]:
using ImageMorphology;

function get_orange_white_boolean(imagen)
    
    diferenciaRojoVerde = 0.3; 
    diferenciaRojoAzul = 0.3;
    
    canalRojo = red.(imagen); 
    canalVerde = green.(imagen); 
    canalAzul = blue.(imagen);
    
    white_chanel = (zeros(size(canalRojo)[1], size(canalRojo)[2]).+0.9);
    white_chanel1 = (zeros(size(canalRojo)[1], size(canalRojo)[2]).+0.9);
    
    mb_w =  (canalRojo.>white_chanel.+ mean(canalRojo)) .& 
            (canalVerde.>white_chanel.- mean(canalVerde)) .& 
            (canalAzul.>white_chanel.- mean(canalAzul));
    
    mb_w1 =  (canalRojo.>white_chanel1) .& 
            (canalVerde.>white_chanel1) .& 
            (canalAzul.>white_chanel1);
    
    mb_r =  (canalRojo.>(canalVerde.+diferenciaRojoVerde)) .& 
            (canalRojo.>(canalAzul.+diferenciaRojoAzul));
    
    matrizBooleana = mb_r .| mb_w .| mb_w1; 
    
    return matrizBooleana;
end;


img = load("fotos/pez001.jpg")
imgd = closing(get_orange_white_boolean(img),[1,2])

labelArray = ImageMorphology.label_components(imgd);

display(Gray.(get_orange_white_boolean(img)))
display(Gray.(imgd))